In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define the set of mathematical operations
# In this example, we'll consider addition, subtraction, multiplication, and division
operations = {
    'add': lambda x, y: x + y,
    'sub': lambda x, y: x - y,
    'mul': lambda x, y: x * y,
    'div': lambda x, y: x / y,
}

# Generate synthetic data
def generate_synthetic_data(num_samples):
    x = np.linspace(-5, 5, num_samples)
    y = x+1
    return x, y

# Convert data to PyTorch tensors
x_data, y_data = generate_synthetic_data(100)

print(x_data)
print(y_data)
x_tensor = torch.tensor(x_data, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.float32)

# Define a simple symbolic regression model
class SymbolicRegressionModel(nn.Module):
    def __init__(self):
        super(SymbolicRegressionModel, self).__init__()
        # Define the trainable coefficients for the operations
        self.coeff_add = nn.Parameter(torch.randn(1))
        self.coeff_sub = nn.Parameter(torch.randn(1))
        self.coeff_mul = nn.Parameter(torch.randn(1))
        self.coeff_div = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        # Apply the symbolic expression to input data
        y_pred = (
            self.coeff_add * operations['add'](x, 1) +
            self.coeff_sub * operations['sub'](x, 2) +
            self.coeff_mul * operations['mul'](x, 3) +
            self.coeff_div * operations['div'](x, 4)
        )
        return y_pred

# Create the model and define a loss function
num_epochs = 5000

model = SymbolicRegressionModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.1)

for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(x_tensor)
    
    # Compute loss
    loss = criterion(y_pred, y_tensor)
    
    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the data
with torch.no_grad():
    y_predicted = model(x_tensor)

# Extract the learned coefficients from the trained model
coeff_addition = model.coeff_add.item()
coeff_subtraction = model.coeff_sub.item()
coeff_multiplication = model.coeff_mul.item()
coeff_division = model.coeff_div.item()

# Define the final symbolic equation
final_equation = f"{coeff_addition}*x + {coeff_subtraction}*2 - {coeff_multiplication}*3*x + {coeff_division}*4"

print(f"Final Equation: {final_equation}")

[-5.         -4.8989899  -4.7979798  -4.6969697  -4.5959596  -4.49494949
 -4.39393939 -4.29292929 -4.19191919 -4.09090909 -3.98989899 -3.88888889
 -3.78787879 -3.68686869 -3.58585859 -3.48484848 -3.38383838 -3.28282828
 -3.18181818 -3.08080808 -2.97979798 -2.87878788 -2.77777778 -2.67676768
 -2.57575758 -2.47474747 -2.37373737 -2.27272727 -2.17171717 -2.07070707
 -1.96969697 -1.86868687 -1.76767677 -1.66666667 -1.56565657 -1.46464646
 -1.36363636 -1.26262626 -1.16161616 -1.06060606 -0.95959596 -0.85858586
 -0.75757576 -0.65656566 -0.55555556 -0.45454545 -0.35353535 -0.25252525
 -0.15151515 -0.05050505  0.05050505  0.15151515  0.25252525  0.35353535
  0.45454545  0.55555556  0.65656566  0.75757576  0.85858586  0.95959596
  1.06060606  1.16161616  1.26262626  1.36363636  1.46464646  1.56565657
  1.66666667  1.76767677  1.86868687  1.96969697  2.07070707  2.17171717
  2.27272727  2.37373737  2.47474747  2.57575758  2.67676768  2.77777778
  2.87878788  2.97979798  3.08080808  3.18181818  3